In [ ]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

## API Overview

This notebook provides a demonstration of how to interact with graphrag as a library using the API as opposed to the CLI. Note that graphrag's CLI actually connects to the library through this API for all operations. 

In [12]:
from pathlib import Path
from pprint import pprint

import pandas as pd

import graphrag.api as api
from graphrag.config.load_config import load_config
from graphrag.index.typing.pipeline_run_result import PipelineRunResult

In [13]:
PROJECT_DIRECTORY = "../../svtest"

## Prerequisite
As a prerequisite to all API operations, a `GraphRagConfig` object is required. It is the primary means to control the behavior of graphrag and can be instantiated from a `settings.yaml` configuration file.

Please refer to the [CLI docs](https://microsoft.github.io/graphrag/cli/#init) for more detailed information on how to generate the `settings.yaml` file.

### Generate a `GraphRagConfig` object

In [16]:
graphrag_config = load_config(Path(PROJECT_DIRECTORY))

## Indexing API

*Indexing* is the process of ingesting raw text data and constructing a knowledge graph. GraphRAG currently supports plaintext (`.txt`) and `.csv` file formats.

## Build an index

In [17]:
index_result: list[PipelineRunResult] = await api.build_index(config=graphrag_config)

# index_result is a list of workflows that make up the indexing pipeline that was run
for workflow_result in index_result:
    status = f"error\n{workflow_result.errors}" if workflow_result.errors else "success"
    print(f"Workflow Name: {workflow_result.workflow}\tStatus: {status}")

Workflow Name: create_base_text_units	Status: success
Workflow Name: create_final_documents	Status: success
Workflow Name: extract_graph	Status: success
Workflow Name: finalize_graph	Status: success
Workflow Name: create_communities	Status: success
Workflow Name: create_final_text_units	Status: success
Workflow Name: create_community_reports	Status: success
Workflow Name: generate_text_embeddings	Status: success


## Query an index

To query an index, several index files must first be read into memory and passed to the query API. 

In [18]:
entities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/entities.parquet")
communities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/communities.parquet")
community_reports = pd.read_parquet(
    f"{PROJECT_DIRECTORY}/output/community_reports.parquet"
)

response, context = await api.global_search(
    config=graphrag_config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    community_level=2,
    dynamic_community_selection=False,
    response_type="Multiple Paragraphs",
    query="Who is apt i2c?",
)

The response object is the official reponse from graphrag while the context object holds various metadata regarding the querying process used to obtain the final response.

In [19]:
print(response)

Based on the provided analyst reports, here is a synthesized response about **APBI2C**:

### Overview of APBI2C  
APBI2C is a module within the **GLADCI organization**, specifically designed to interface with **I²C (Inter-Integrated Circuit) protocols**. Its primary role is to facilitate communication within the system, acting as a bridge for data exchange between components that use the I²C standard [Data: Reports (5, 11)].  

### Key Functionality and Signals  
The module's operation relies on critical signals, including:  
- **PCLK**: Serves as the system clock input, synchronizing operations.  
- **PRESETN**: An active-low reset signal, essential for initializing or resetting the module.  
These signals are fundamental to ensuring proper functionality and stability [Data: Reports (11)].  

### Structural Representation  
A visual depiction of the APBI2C module is provided in **FIGURE 1**, which illustrates its top-level block structure. This diagram helps clarify how the module int

Digging into the context a bit more provides users with extremely granular information such as what sources of data (down to the level of text chunks) were ultimately retrieved and used as part of the context sent to the LLM model).

In [7]:
pprint(context)  # noqa: T203

{'reports':     id                                              title  occurrence weight  \
0    3           Dulce Team and Paranormal Military Squad           1.000000   
1    1  Paranormal Military Squad and Interstellar Com...           0.638655   
2   47       Dulce Base and the Paranormal Military Squad           0.554622   
3   27       Dulce Base and the Paranormal Military Squad           0.554622   
4   33  Paranormal Military Squad and Dulce Base Opera...           0.529412   
5   32       Dulce Base and the Paranormal Military Squad           0.344538   
6   23          Dulce Team and Interstellar Communication           0.319328   
7   41   Taylor and the Interplanetary Communication Team           0.285714   
8   10           Dulce Base and Paranormal Military Squad           0.277311   
9   25  Dulce Base and Extraterrestrial Communications...           0.243697   
10  46  Dr. Jordan Hayes and the Extraterrestrial Comm...           0.243697   
11  38  TEAM and the Alien C